In [1]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib

import pandas as pd
import numpy as np

from scipy.io import wavfile
from scipy import signal

import torch
import torchvision
import torchvision.transforms as transforms

FRAME_SIZE = 2048
HOP_SIZE = 512
# lughezza in secondi che deve avere il singolo frame
CHUNK_SIZE = 1.2
# valore di ampizza sotto il quale il segnale viene considerato silenzio
SILENCE_THRESHOLD = 0.01
# directory di output per gli spettrogrammi
OUTPUT_DIR = Path("spectrograms")

In [2]:
def silece_detection(data, threshold=SILENCE_THRESHOLD):
    # controllo se il chunk contiene solo silenzio. 
    silence = np.abs(data) < threshold
    if np.all(silence):
        return None
    return data

def chunk_data(data, sample_rate, chunk_size):
    # calcolo il numero di campioni per il chunk
    chunk_samples = int(sample_rate * chunk_size)

    # creazione dei chunk
    chunks = [data[i:i + chunk_samples] for i in range(0, len(data), chunk_samples) if i + chunk_samples <= len(data)]
    # controllo se l'ultimo chunk ha lasciato una porzione di audio
    if len(data) % chunk_samples != 0:
        chunks.append(data[len(data) - (len(data) % chunk_samples):])
        
    return chunks

def min_max_norm(data):
    max_val = np.max(data)
    min_val = np.min(data)
    denom = max_val - min_val if max_val != min_val else 1e-10
    return (2*(data - min_val)/(denom))-1

def save_chunk(ax, fig, data, file_name):
    # salvataggio dello spettrogramma
    ax.clear()
    im = ax.imshow(data, aspect='auto', origin='lower', cmap='magma')

    category = file_name.split('-')[-1]
    filename = file_name + ".png" 
    category_dir = OUTPUT_DIR / category

    fig.savefig(category_dir / filename, bbox_inches='tight')
    plt.close(fig)



In [3]:
i = 0
def preprocessing(full_data, sample, file_name):
    matplotlib.use('Agg')
    fig, ax = plt.subplots(figsize=(10, 4))

    # definizione della finestra di Hann
    hann_win = signal.windows.hann(FRAME_SIZE)

    # controllo se il segnale è campionato nel modo corretto
    if sample[0] != sample[1]:
        full_data = signal.resample(full_data, sample[0])

    # il segnale audio viene diviso in chunk
    chunked_data = chunk_data(full_data, sample[0], CHUNK_SIZE)
    chunk_num = 0
    for data in chunked_data:
        # elimino i chunk che contengono solo silenzio
        data = silece_detection(data)
        if data is None:
            continue
        
        # normalizzazione segnale min-max tra -1 e 1
        data_norm = min_max_norm(data)
        
        # calcolo dell'STF
        SFT = signal.ShortTimeFFT(hann_win, hop=HOP_SIZE, fs=FRAME_SIZE)
        s_x = SFT.stft(data_norm)

        # calcolo valori per lo spettrogramma, viene ritornata la versione logaritmica dei valori
        spectrogram = np.abs(s_x)**2

        # normalizzazione dello spettrogramma
        log_spectrogram = np.log(spectrogram + 1e-10)

        # salvataggio del chunk dello spettrogrammma in formato PNG
        composed_filename = str(chunk_num)+'-'+'-'.join(file_name.split('-')[1:])
        save_chunk(ax, fig, log_spectrogram, composed_filename)

        chunk_num += 1
    global i
    i += 1
    print(f"Processed {i} files")



In [4]:
audio_files = os.listdir(Path('ESC-50') / 'audio')
output_dir = Path("spectrograms")

# lettura del dataset e drop delle colonne non necessarie
class_df = pd.read_csv(Path('ESC-50') / 'esc50.csv')
class_df = class_df.drop(columns=['fold', 'esc10', 'src_file', 'take'])

# elenco classi
classes = class_df['target'].unique().astype(str)

# creazione delle cartelle di output divise per classe
for category in classes:
    category_dir = output_dir / category
    category_dir.mkdir(parents=True, exist_ok=True)

In [5]:
# lettura dei file e salvataggio in un dataframe con il nome del file, la categoria e i dati audio
audio_list = []
for file in audio_files:
    # lettura dati audio
    sample, data = wavfile.read(Path('ESC-50') / 'audio' / file)
    data = data.astype(np.float32)

    # estrazione della nome del file
    file_path = Path(file).stem 
    
    audio_list.append((file_path, sample, data))
audio_df = pd.DataFrame(audio_list, columns=['filename', 'sample_rate', 'data']) 

# estrazione del valore del rate di campionamento più frequente. In futuro variabile globale univoca
standard_sample = audio_df['sample_rate'].mode()[0]

# chiamata sulla colonna data della funzione di preprocessing
audio_df.apply(lambda row: preprocessing(row['data'], (standard_sample, row['sample_rate']), row['filename']), axis=1)

Processed 1 files
Processed 2 files
Processed 3 files
Processed 4 files
Processed 5 files
Processed 6 files
Processed 7 files
Processed 8 files
Processed 9 files
Processed 10 files
Processed 11 files
Processed 12 files
Processed 13 files
Processed 14 files
Processed 15 files
Processed 16 files
Processed 17 files
Processed 18 files
Processed 19 files
Processed 20 files
Processed 21 files
Processed 22 files
Processed 23 files
Processed 24 files
Processed 25 files
Processed 26 files
Processed 27 files
Processed 28 files
Processed 29 files
Processed 30 files
Processed 31 files
Processed 32 files
Processed 33 files
Processed 34 files
Processed 35 files
Processed 36 files
Processed 37 files
Processed 38 files
Processed 39 files
Processed 40 files
Processed 41 files
Processed 42 files
Processed 43 files
Processed 44 files
Processed 45 files
Processed 46 files
Processed 47 files
Processed 48 files
Processed 49 files
Processed 50 files
Processed 51 files
Processed 52 files
Processed 53 files
Pr

0       None
1       None
2       None
3       None
4       None
        ... 
1995    None
1996    None
1997    None
1998    None
1999    None
Length: 2000, dtype: object